In [ ]:
#Autoreload to be able to tweak source code in the fly without restarting kernel
%load_ext autoreload

In [ ]:
%autoreload 2

In [1]:
import aligater as ag

In [ ]:
fcsList=ag.collectFiles("/media/ludvig/Project_Storage/GAIM/data/GAIM_fcs/", lFilter=["B cell"])

In [ ]:
fcsDF = ag.loadFCS(fcsList[1600], compensate=True)

In [ ]:
no_clutter=ag.gateThreshold(fcsDF,"FSC-A", "FSC-H",thresh=260000, orientation='horisontal',population="lower")

In [ ]:
singlets=ag.gatePC(fcsDF,"FSC-A", "FSC-H",center='density',widthScale=4, heightScale=4, vI=no_clutter, plot=True)

In [ ]:
PBMCstep1=ag.gateThreshold(fcsDF,xCol="FSC-A",yCol="SSC-A",orientation="vertical",thresh=70000,population="lower")

In [ ]:
PBMC=ag.gatePC(fcsDF,"FSC-A","SSC-A",center='centroid',widthScale=3, heightScale=2, vI=PBMCstep1, plot=True)

In [ ]:
mean, sigma, maxVal=ag.axisStats(fcsDF, "CD34", PBMC)
sigma*3

In [ ]:
CD45step1 = ag.gateThreshold(fcsDF, "CD34", "CD45", orientation='horisontal', vI=PBMC,population="lower",thresh=2*sigma, scale='logish')

In [ ]:
lim = ag.valleySeek(fcsDF, "CD45",vI=CD45step1,interval=[0,2500],sigma=2,bins=1000)

In [ ]:
CD45pos = ag.gateThreshold(fcsDF, "CD34", "CD45", vI=CD45step1, orientation='vertical',thresh=lim,scale='logish') 

In [ ]:
CD34step1=ag.gateThreshold(fcsDF, "CD34", "CD45", orientation='horisontal', vI=PBMC,population="upper",thresh=(2*sigma), scale='logish')

In [ ]:
ag.plot_densityFunc(fcsDF,"CD45", vI=CD34step1,sigma=2,bins=200)

In [ ]:
lim=ag.valleySeek(fcsDF, "CD45",vI=CD34step1,interval=[500,2500],sigma=2,bins=200)
lim

In [ ]:
CD34pos=ag.gateThreshold(fcsDF, "CD34", "CD45", orientation='vertical', vI=CD34step1,population="lower",thresh=lim, scale='logish')

In [ ]:
ag.plotHeatmap(fcsDF, "CD19","CD45",vI=CD45pos,scale='logish')

In [ ]:
ag.plot_densityFunc(fcsDF,xCol="CD19",vI=CD45pos, bins=1000, sigma=2, scale='logish')

In [ ]:
lim=ag.valleySeek(fcsDF,xCol="CD19",vI=CD45pos,interval=[0,2000],bins=1000,sigma=2, scale='logish')

In [ ]:
lim

In [ ]:
CD19pos=ag.gateThreshold(fcsDF,"CD19","CD45",vI=CD45pos,thresh=lim,scale='logish',orientation='horisontal',population='upper')

In [ ]:
ag.plotHeatmap(fcsDF, "IgD", "CD27", vI=CD19pos,scale="logish",bins=300)

In [ ]:
ag.plot_densityFunc(fcsDF,"IgD", CD19pos,bins=300,sigma=2, scale='logish')

In [ ]:
xlim=ag.valleySeek(fcsDF,"IgD", CD19pos, interval=[0,4000],bins=300,sigma=2, scale='logish')

In [ ]:
ag.plot_densityFunc(fcsDF,"CD27", CD19pos,bins=500,sigma=1, scale='logish')

In [ ]:
ylim=ag.valleySeek(fcsDF,"CD27", CD19pos, interval=[0,4000],bins=300,sigma=2, scale='logish')

In [ ]:
switchB, preSwitchB, naiveB, exhaustedB = ag.quadGate(fcsDF,"IgD","CD27",vI=CD19pos,xThresh=xlim,yThresh=ylim, scale='logish')

In [ ]:
mean, sigma, maxVal = ag.axisStats(fcsDF, "CD38", vI=switchB)

In [ ]:
ag.plot_densityFunc(fcsDF,"CD38",switchB,scale='logish',bins=300,sigma=2)

In [ ]:
plasmablasts=ag.gateCorner(fcsDF, "CD24", "CD38",1000,mean+sigma,"lower","upper", vI=switchB,scale='logish')

In [ ]:
xmean,xsigma,xmaxVal = ag.axisStats(fcsDF,xCol="CD24",vI=naiveB)

In [ ]:
ymean,ysigma,ymaxVal = ag.axisStats(fcsDF,xCol="CD38",vI=naiveB)

In [ ]:
transitionals=ag.gateCorner(fcsDF,"CD24", "CD38",xThresh = xmaxVal, yThresh=ymaxVal+2000, vI=naiveB, scale='logish')

In [ ]:
lim = ag.valleySeek(fcsDF,"IgA",vI=switchB,interval=[750,2000],bins=300, sigma=0, scale='logish')
lim

In [ ]:
IgApos=ag.gateThreshold(fcsDF, "IgA","CD19", vI=switchB, thresh=lim, scale='logish')